In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import folium

In [ ]:
# Read the CSV file into a DataFrame
df_final = pd.read_csv("List.csv")

In [ ]:
# Ensure 'lattitude' and 'longitude' columns are present
if 'lattitude' not in df_final.columns or 'longitude' not in df_final.columns:
    raise ValueError("DataFrame must contain 'lattitude' and 'longitude' columns.")

In [ ]:
# Prepare data for clustering
X = df_final[['lattitude', 'longitude']].values
max_k = 5
distortions = []

In [ ]:
# Calculate distortions for different values of k
for i in range(1, max_k + 1):
    if len(X) >= i:
        model = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        model.fit(X)
        distortions.append(model.inertia_)

In [ ]:
# Determine the best k using the "elbow" method
if len(distortions) >= 3:
    second_derivatives = np.diff(np.diff(distortions))
    best_k_index = np.argmin(second_derivatives) + 2  # +2 due to diff reduction
    best_k = best_k_index
else:
    best_k = 3  # Default value if not enough data

In [ ]:
# Run K-means clustering on the latitude and longitude columns
kmeans = KMeans(n_clusters=best_k, random_state=0).fit(X)
df_final['Cluster'] = kmeans.labels_.astype(str)
df_final.to_csv('List.csv', index=False)

In [ ]:
# Define coordinates for the center of the map
map_center = [17.493527, 78.391525]
map_bom = folium.Map(location=map_center, zoom_start=12)

In [ ]:
# Define a color scheme for clusters
def color_producer(cluster):
    colors = ['orange', 'green', 'blue', 'red']  # Ensure there are enough colors for your clusters
    return colors[int(cluster) % len(colors)]

In [ ]:
# Plotting clustered locations on the map
for _, row in df_final.iterrows():
    folium.CircleMarker(
        location=[row['lattitude'], row['longitude']],
        fill=True,
        fill_opacity=1,
        popup=folium.Popup(row['Cluster'], max_width=300),
        radius=5,
        color=color_producer(row['Cluster'])
    ).add_to(map_bom)

In [ ]:
# Add a marker for JNTU
folium.Marker(map_center, popup='JNTU').add_to(map_bom)

In [ ]:
# Save the map to an HTML file
map_bom.save("clustered_map.html")